In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy import stats 


### 只保留有數字的薪資，排除面議的

In [ ]:
df=pd.read_csv(r"C:\Users\FM_pc\Desktop\暫時匯出查看.csv")
df_filter=df[df['薪資是否為面議']==0].copy()

In [ ]:
df_filter['要求python']=df_filter['專長工具'].fillna('').str.contains('python',case=False)

In [ ]:
df_filter.tail(5)

### 分組薪資資料

In [ ]:
with_py=df_filter[df_filter['要求python']]['薪資下限']
without_py=df_filter[~df_filter['要求python']]['薪資下限']

### 要求python描述統計

In [ ]:
with_py.describe()

### 不要求python描述統計

In [ ]:
without_py.describe()

### 繪製箱型圖，判斷資料是否偏態、觀察極端值

In [ ]:
plt.figure(figsize=(12,5))
plt.rcParams['font.family'] = 'Microsoft JhengHei'
plt.rcParams['axes.unicode_minus'] = False
sns.boxplot(x='要求python',y='薪資下限',data=df_filter)
plt.title('薪資下限的箱型圖(是否要求python)')

### 用IQR(InterQuantile Range)去除極端值

In [ ]:
def remove_outlier(group):
    Q1=group['薪資下限'].quantile(0.25)
    Q3=group['薪資下限'].quantile(0.75)
    IQR=Q3-Q1
    lower_bound=Q1-1.5*IQR
    upper_bound=Q3+1.5*IQR
    return group[(group['薪資下限']>=lower_bound ) & (group['薪資下限']<=upper_bound)]

df_no_outlier=df_filter.groupby('要求python',group_keys=False).apply(remove_outlier)

### 去除極端值後的箱型圖

In [ ]:
sns.boxplot(x='要求python',y="薪資下限",data=df_no_outlier)
plt.title("薪資下限去除極端值的箱型圖(是否要求python)")

### 因有明顯右偏態，故取自然對數，使資料分布更接近常態分佈。

In [ ]:
df_no_outlier['log_薪資下限']=np.log(df_no_outlier['薪資下限'])

with_py_log=df_no_outlier[df_no_outlier['要求python']]['log_薪資下限']
without_py_log=df_no_outlier[~df_no_outlier['要求python']]['log_薪資下限']

### 檢查常態性(shapiro-wilk)

In [ ]:
stats.shapiro(with_py_log)

In [ ]:
stats.shapiro(without_py_log)

### 檢查變異數齊性(levene's test)

In [ ]:
stats.levene(with_py_log,without_py_log)

### 因資料不符合 t-test 的前提（常態性與變異數齊性），故改用非參數檢定 Mann-Whitney U Test。該檢定不要求資料常態分布與變異數齊性，適用於比較兩組樣本的中位數是否存在顯著差異。

In [ ]:
with_py_no_outlier=df_no_outlier[df_no_outlier['要求python']]['薪資下限']
without_py_no_outlier=df_no_outlier[~df_no_outlier['要求python']]['薪資下限']
u_stat,p_value=stats.mannwhitneyu(with_py_no_outlier,without_py_no_outlier,alternative="two-sided")
print(f"Mann-Whitneyu U test 統計量:{u_stat}")
print(f"p-value:{p_value}")

In [ ]:
df_no_outlier['要求python'].value_counts()